In [5]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report ,confusion_matrix,accuracy_score
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, ReLU ,BatchNormalization, Dropout ,MaxPooling1D, Conv1D,Flatten,LSTM
from keras.optimizers import Adam
import pickle
from keras.preprocessing.text import Tokenizer

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/silverline/MirrorMind/function2/function2_quiz.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    130 non-null    object
 1   status  130 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.2+ KB


In [ ]:
df.head()

,text,status
0,Have you noticed any differences in how you ex...,1
1,Have you noticed any differences in how you ex...,1
2,Have you noticed any differences in how you ex...,1
3,Have you noticed any differences in how you ex...,1
4,Have you noticed any differences in how you ex...,1


In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download NLTK resources if not already downloaded
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    words = nltk.word_tokenize(text)  # Tokenize the text into words
    words = [lemmatizer.lemmatize(word) for word in words]  # Lemmatize words
    words = [word for word in words if word.lower() not in stop_words]  # Remove stop words
    return ' '.join(words)




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df['prepros_text'] = df['text'].apply(preprocess_text)
df['prepros_text'][0]

In [ ]:
df.head()

,text,status,prepros_text
0,Have you noticed any differences in how you ex...,1,noticed difference express emotion facial expr...
1,Have you noticed any differences in how you ex...,1,noticed difference express emotion facial expr...
2,Have you noticed any differences in how you ex...,1,noticed difference express emotion facial expr...
3,Have you noticed any differences in how you ex...,1,noticed difference express emotion facial expr...
4,Have you noticed any differences in how you ex...,1,noticed difference express emotion facial expr...


In [ ]:
df.drop(columns=['text'],axis=1,inplace=True)
df.to_csv('/content/drive/MyDrive/silverline/MirrorMind/function2/function2_quiz_pros.csv')

## Build Model

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/silverline/MirrorMind/function2/function2_quiz_pros.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    130 non-null    int64 
 1   status        130 non-null    int64 
 2   prepros_text  130 non-null    object
dtypes: int64(2), object(1)
memory usage: 3.2+ KB


In [ ]:
df.head()

,Unnamed: 0,status,prepros_text
0,0,1,noticed difference express emotion facial expr...
1,1,1,noticed difference express emotion facial expr...
2,2,1,noticed difference express emotion facial expr...
3,3,1,noticed difference express emotion facial expr...
4,4,1,noticed difference express emotion facial expr...


In [ ]:
len(df['prepros_text'].iloc[96].split())

17

In [ ]:
checkpoint=ModelCheckpoint(
    '/content/drive/MyDrive/silverline/MirrorMind/function2/function2_quiz_model.h5',
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
)
stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    mode='min',
)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['prepros_text'])
sequences = tokenizer.texts_to_sequences(df['prepros_text'])
X= pad_sequences(sequences, maxlen=25)

with open('/content/drive/MyDrive/silverline/MirrorMind/function2/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

y=df['status']

In [ ]:
X[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   6,  42,  36,   3,   2,
         1,  45,   7,   6, 187,  36,  46, 188,  10,   4,  76,  11],
      dtype=int32)

In [ ]:
model = tf.keras.models.Sequential([
    Conv1D(256, kernel_size=2, input_shape=(X.shape[1], 1)),
    BatchNormalization(),
    ReLU(),
    MaxPooling1D(pool_size=2),
    Conv1D(512, kernel_size=3),
    BatchNormalization(),
    ReLU(),
    MaxPooling1D(pool_size=2),
    Conv1D(512, kernel_size=3),
    BatchNormalization(),
    ReLU(),
    MaxPooling1D(pool_size=2),
    LSTM(512,return_sequences=True),
    LSTM(1024),
    Flatten(),
    Dense(1024),
    BatchNormalization(),
    ReLU(),
    Dropout(0.5),
    Dense(512),
    BatchNormalization(),
    ReLU(),
    Dense(256),
    BatchNormalization(),
    ReLU(),
    Dropout(0.5),
    Dense(64),
    BatchNormalization(),
    ReLU(),
    Dense(1, activation='sigmoid')
])


model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_18 (Conv1D)          (None, 24, 256)           768       
                                                                 
 batch_normalization_42 (Bat  (None, 24, 256)          1024      
 chNormalization)                                                
                                                                 
 re_lu_42 (ReLU)             (None, 24, 256)           0         
                                                                 
 max_pooling1d_18 (MaxPoolin  (None, 12, 256)          0         
 g1D)                                                            
                                                                 
 conv1d_19 (Conv1D)          (None, 10, 512)           393728    
                                                                 
 batch_normalization_43 (Bat  (None, 10, 512)         

In [ ]:
model.fit(X, y, epochs=500, batch_size=32,callbacks=[checkpoint],validation_split=0.1)

Epoch 1/500
4/4 [==============================] - 7s 589ms/step - loss: 0.7275 - accuracy: 0.5299 - val_loss: 0.6944 - val_accuracy: 0.3077
Epoch 2/500
4/4 [==============================] - 0s 25ms/step - loss: 0.6631 - accuracy: 0.6410 - val_loss: 0.6942 - val_accuracy: 0.3077
Epoch 3/500
4/4 [==============================] - 0s 28ms/step - loss: 0.6331 - accuracy: 0.6838 - val_loss: 0.6954 - val_accuracy: 0.3077
Epoch 4/500
4/4 [==============================] - 0s 24ms/step - loss: 0.5673 - accuracy: 0.7009 - val_loss: 0.6967 - val_accuracy: 0.3077
Epoch 5/500
4/4 [==============================] - 0s 27ms/step - loss: 0.5341 - accuracy: 0.7265 - val_loss: 0.6977 - val_accuracy: 0.3077
Epoch 6/500
4/4 [==============================] - 0s 29ms/step - loss: 0.5046 - accuracy: 0.7778 - val_loss: 0.6978 - val_accuracy: 0.3077
Epoch 7/500
4/4 [==============================] - 0s 34ms/step - loss: 0.6107 - accuracy: 0.7009 - val_loss: 0.6985 - val_accuracy: 0.3077
Epoch 8/500
4/4 [==

In [6]:
from keras.models import load_model

test="Are there times when you feel like your facial expressions lead to misunderstandings Not really. I think my expressions align with how I'm feeling, so misunderstandings are rare"
test=preprocess_text(test)
with open('/content/drive/MyDrive/silverline/MirrorMind/function2/tokenizer.pickle', 'rb') as handle:
  loaded_tokenizer = pickle.load(handle)
  sequences = loaded_tokenizer.texts_to_sequences([test])
  X_t=pad_sequences(sequences, maxlen=25)
  # print(X_t[0])
model=load_model('/content/drive/MyDrive/silverline/MirrorMind/function2/function2_quiz_model.h5')
result=model.predict(X_t)
result

1/1 [==============================] - 1s 1s/step


array([[0.06890146]], dtype=float32)

In [7]:
test="Are there times when you feel like your facial expressions lead to misunderstandings Definitely. There have been times when people thought I was angry when I wasn't. It's like my face betrays how I feel"
test=preprocess_text(test)
with open('/content/drive/MyDrive/silverline/MirrorMind/function2/tokenizer.pickle', 'rb') as handle:
  loaded_tokenizer = pickle.load(handle)
  sequences = loaded_tokenizer.texts_to_sequences([test])
  X_t=pad_sequences(sequences, maxlen=25)
  # print(X_t[0])
model=load_model('/content/drive/MyDrive/silverline/MirrorMind/function2/function2_quiz_model.h5')
result=model.predict(X_t)
result

1/1 [==============================] - 1s 994ms/step


array([[0.9818727]], dtype=float32)